In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten , Input, Dropout, concatenate
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import Embedding

In [3]:
# Load data
df = pd.read_csv('labeled_data.csv')
#,count,hate_speech,offensive_language,neither,class,tweet
# Preprocess data

In [4]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
text = df['tweet']
label = df['class']

In [6]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence_length)

In [7]:
# Load GloVe word embeddings
embedding_dim = 100
word_vectors = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_vectors[word] = coefs

ValueError: could not convert string to float: 'n'

In [231]:
# Create embedding matrix
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = word_vectors.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [232]:
y=pd.get_dummies(labels.values)
#[print(data[labels[i],y[i] for i in range(0,5)]

In [233]:
from sklearn.model_selection import train_test_split

In [234]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size = 0.2)

In [235]:
x_train.shape

(19826, 100)

In [236]:
embedding_dim = 100

In [237]:
# Load pre-trained Word2Vec model.
#model = gensim.models.Word2Vec.load('Aoword2vec.bin')
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('Aoword2vec.bin', binary=True)

# Define parameters
max_sequence_length = 100
num_classes = 3
embedding_dim = 100
num_filters = 32
dropout_prob = 0.5
hidden_dims = 64
lr = 0.01
batch_size = 128
num_epochs = 2

In [238]:
from keras.models import Model

In [239]:
# Define CNN model
inputs = Input(shape=(max_sequence_length,), dtype='int32')
embedding = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True)(inputs)
conv_0 = Conv1D(num_filters, 3, activation='relu')(embedding)
maxpool_0 = MaxPooling1D(max_sequence_length - 3 + 1)(conv_0)
dropout_0 = Dropout(dropout_prob)(maxpool_0)
conv_1 = Conv1D(num_filters, 4, activation='relu')(embedding)
maxpool_1 = MaxPooling1D(max_sequence_length - 4 + 1)(conv_1)
dropout_1 = Dropout(dropout_prob)(maxpool_1)
conv_2 = Conv1D(num_filters, 5, activation='relu')(embedding)
maxpool_2 = MaxPooling1D(max_sequence_length - 5 + 1)(conv_2)
dropout_2 = Dropout(dropout_prob)(maxpool_2)
merged_tensor = concatenate([dropout_0, dropout_1, dropout_2], axis=1)
flatten = Flatten()(merged_tensor)
dense = Dense(hidden_dims, activation='relu')(flatten)
dropout = Dropout(dropout_prob)(dense)
output = Dense(num_classes, activation='sigmoid')(dropout)
cnnmodel = Model(inputs, output)

In [240]:
cnnmodel.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 embedding_13 (Embedding)       (None, 100, 100)     3650900     ['input_17[0][0]']               
                                                                                                  
 conv1d_41 (Conv1D)             (None, 98, 32)       9632        ['embedding_13[0][0]']           
                                                                                                  
 conv1d_42 (Conv1D)             (None, 97, 32)       12832       ['embedding_13[0][0]']           
                                                                                            

In [241]:
# Train the CNN model
cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnnmodel.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
620/620 [==============================] - 147s 111ms/step - loss: 0.5635 - accuracy: 0.8058 - val_loss: 0.3325 - val_accuracy: 0.8820
Epoch 2/5
620/620 [==============================] - 52s 85ms/step - loss: 0.3476 - accuracy: 0.8866 - val_loss: 0.2962 - val_accuracy: 0.8955
Epoch 3/5
620/620 [==============================] - 55s 88ms/step - loss: 0.2828 - accuracy: 0.9089 - val_loss: 0.2772 - val_accuracy: 0.9001
Epoch 4/5
620/620 [==============================] - 54s 87ms/step - loss: 0.2424 - accuracy: 0.9181 - val_loss: 0.2883 - val_accuracy: 0.8961
Epoch 5/5
620/620 [==============================] - 53s 86ms/step - loss: 0.2050 - accuracy: 0.9304 - val_loss: 0.2904 - val_accuracy: 0.8993


In [258]:
# Create the LR model
lr_model = LogisticRegression()
y_train = y_train.ravel()
x_train = x_train.ravel().reshape(-1,1)
# Train the LR model
lr_model.fit(x_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1982600, 59478]

In [ ]:
# Use the CNN and LR models to make predictions on the validation set
cnn_preds = cnn_model.predict(val_sequences)
lr_preds = lr_model.predict(val_sequences)
ensemble_preds = []
for cnn_pred, lr_pred in zip(cnn_preds, lr_preds):
    if cnn_pred.argmax() == lr_pred.argmax():
        ensemble_preds.append(cnn_pred)
    else:
        ensemble_preds.append((cnn_pred + lr_pred) / 2)

# Evaluate the ensemble model on the validation set
ensemble_preds = np.array(ensemble_preds)
ensemble_acc = np.mean(np.argmax(ensemble_preds, axis=1) == np.argmax(val_labels, axis=1))
print('Ensemble Accuracy:', ensemble_acc)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Load the IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

# Pad sequences to a maximum review length of 500 words
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# Create the Logistic Regression model
model = Sequential()
model.add(Embedding(5000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model on the training data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128)

# Evaluate the model on the test data
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))